## Retrieve Congress Bill Info- Congress.Gov API

#### Create a virtual environment to retrieve the congressional legislation

In [ ]:
!python3 -m venv congress-env
!source congress-env/bin/activate
!python3 -m pip install --upgrade pip
!git clone https://github.com/unitedstates/congress.git

In [ ]:
cd congress

In [ ]:
!pip install .

In [ ]:
!usc-run votes --congress=118 --session=2024

#### Retrieve Bill information from Congress.Gov

In [ ]:
import requests
import re
import ast
import operator
import os, shutil
import urllib.request
import pickle
import time
import sys
import gzip
import os
import json

Use the Congress Gov API Key to retrieve bills from the website.

The following function extracts general bill info:

In [ ]:
def extract_bill_info(Congress, Number, BillType, API_RETURN_COUNT):
    api_key = 'YOUR_API_KEY'
    headers = {
        'X-API-Key': '4V0pWr1YRKYrjzzqYGVjAMrqeTMiDT4x86HxsUXi'
    }

    # Construct the request URL
    endpoint = 'https://api.congress.gov/v3/bill/' + str(Congress) + '/' + BillType + '/' + str(Number)
    params = {
        'limit': 1,  # Adjust as needed
        'offset': 0   # Adjust as needed
    }

    # Make the request
    response = requests.get(endpoint, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json() # or response.xml() based on the expected format
            UpdatedDate = data['bill']['latestAction']['actionDate']
            IntroducedDate = data['bill']['introducedDate']
            Party = data['bill']['sponsors'][0]['party']
            Chamber = data['bill']['originChamber']
            Title = data['bill']['title']
            policyArea = data['bill']['policyArea']['name']
        except:
            UpdatedDate = ''
            IntroducedDate = ''
            Party = ''
            Chamber = ''
            policyArea = ''
            Title = ''
    else:
        print("Error:", response.status_code)
        print(API_RETURN_COUNT)
        UpdatedDate = ''
        IntroducedDate = ''
        Party = ''
        Chamber = ''
        policyArea = ''
        Title = ''
    
    return UpdatedDate,IntroducedDate,Party,Chamber, policyArea, Title

The following function extracts summary info:

In [ ]:
def retrieve_summary(Congress, Number, BillType, API_RETURN_COUNT):
    api_key = 'YOUR_API_KEY'
    headers = {
        'X-API-Key': '4V0pWr1YRKYrjzzqYGVjAMrqeTMiDT4x86HxsUXi'
    }
    SummaryURL = "https://api.congress.gov/v3/bill/"+ str(Congress) + '/' + BillType + '/' + str(Number) + "/summaries"
    # Construct the request URL
    endpoint = SummaryURL
    params = {
        'limit': 1,  # Adjust as needed
        'offset': 0   # Adjust as needed
    }

    # Make the request
    response = requests.get(endpoint, headers=headers, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()  # or response.xml() based on the expected format
            Summary = data['summaries'][0]['text']
        except:
            Summary = ''
    else:
        print("Error:", response.status_code)
        print(API_RETURN_COUNT)
        Summary = ''
    
    return Summary

Extract the link url to the pdf version of the bill

In [ ]:
def retrieve_pdf_url(Congress,Number,BillType,API_RETURN_COUNT):
    api_key = 'YOUR_API_KEY'
    headers = {
        'X-API-Key': '4V0pWr1YRKYrjzzqYGVjAMrqeTMiDT4x86HxsUXi'
    }

    # Construct the request URL
    endpoint = "https://api.congress.gov/v3/bill/"+ str(Congress) + '/' + BillType + '/' + str(Number) + "/text"
    params = {
        'limit': 1,  # Adjust as needed
        'offset': 0   # Adjust as needed
    }

    # Make the request
    response = requests.get(endpoint, headers=headers, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()  # or response.xml() based on the expected format
            pdfurl = data['textVersions'][0]['formats'][1]['url']
        except:
            pdfurl = ''
    else:
        print("Error:", response.status_code)
        print(API_RETURN_COUNT)
        pdfurl = ''
    
    return pdfurl

This is the Class for storing information

In [ ]:
class CongressionalBillInfo:
    def __init__ (self, ID, Title, Summary, IntroducedDate, UpdatedDate, PDF_URL, Party, PolicyArea, Category, Session, Chamber,OriginChamber, Days, Aye, Nay, Not_Voting, Present, DemocratAyeCount, RepublicanAyeCount, RepublicanNayCount, DemocratNayCount, DemocratNotVotingCount, RepublicanNotVotingCount, Result):
        self.ID = ID
        self.Title = Title
        self.Summary = Summary
        self.IntroducedDate = IntroducedDate
        self.UpdatedDate = UpdatedDate
        self.PDF_URL = PDF_URL
        self.Party = Party
        self.PolicyArea = PolicyArea
        self.Category = Category
        self.Chamber = Chamber
        self.OriginChamber = OriginChamber
        self.Session = Session
        self.Days = Days
        self.Aye = Aye
        self.Nay = Nay
        self.Not_Voting = Not_Voting
        self.Present = Present
        self.DemocratAyeCount = DemocratAyeCount
        self.RepublicanAyeCount = RepublicanAyeCount
        self.DemocratNayCount = DemocratNayCount
        self.RepublicanNayCount = RepublicanNayCount
        self.DemocratNotVotingCount = DemocratNotVotingCount
        self.RepublicanNotVotingCount = RepublicanNotVotingCount
        self.Result = Result

In [ ]:
def Adjust_And_Pause():
    time.sleep(3660)
    return 0

In [ ]:
# Replace 'your_directory_path' with the path to your directory
directory_path = 'data/118/votes/2024/'
vote_list = []
# Walk through the directory
for root, dirs, files in os.walk(directory_path):
    for name in dirs:
        file_path = directory_path + name + "/data.json"
        try:
            with open(file_path, 'r') as FILE:
                data = json.load(FILE)
                vote_list.append(data)
        except:
            pass

In [ ]:
print(len(vote_list))

In [ ]:
API_RETURN_COUNT = 0

In [ ]:
from datetime import datetime, timezone
import pytz

BillOBJList = []
counter = 0
for vote in vote_list:
    
    try:
        try:
            Congress = vote['bill']['congress']
        except:
            Congress = vote['congress']

        Session = vote['session']

        try:
            Number = vote['bill']['number']
        except:
            Number = vote['nomination']['number']

        Chamber = vote['chamber']

        try:
            BillType = vote['bill']['type']
        except:
            BillType = "Nomination"

        Category = vote['category'] 
        Result = vote['result']  

        DemocratAyeCount = 0
        RepublicanAyeCount = 0
        IndependentAyeCount = 0

        DemocratNayCount = 0
        RepublicanNayCount = 0
        IndependentNayCount = 0

        DemocratNotVotingCount = 0
        RepublicanNotVotingCount = 0
        IndependentNotVotingCount = 0


        try:
            Aye = vote['votes']['Yea']
        except:
            Aye = vote['votes']['Aye']

        aye_list = []
        for aye_vote in Aye:
            aye_list.append(aye_vote['id'])
            # if aye_vote['party'] == 'D':
            #     DemocratAyeCount+=1
            # elif aye_vote['party'] == 'R':
            #     RepublicanAyeCount+=1

        try:
            Nay = vote['votes']['Nay']
        except:
            Nay = vote['votes']['No']

        nay_list = []
        for nay_vote in Nay:
            nay_list.append(nay_vote['id'])
            # if aye_vote['party'] == 'D':
            #     DemocratNayCount+=1
            # elif aye_vote['party'] == 'R':
            #     RepublicanNayCount+=1

        Not_Voting = vote['votes']['Not Voting']
        not_voting_list = []
        for not_vote in Not_Voting:
            not_voting_list.append(not_vote['id'])
            # if aye_vote['party'] == 'D':
            #     DemocratNotVotingCount+=1
            # elif aye_vote['party'] == 'R':
            #     RepublicanNotVotingCount+=1

        Present = vote['votes']['Present']
        present_list = []
        for pres in Present:
            present_list.append(pres['id'])

        date_str = vote['date']
        # Parsing the date string to a datetime object
        input_date = datetime.fromisoformat(date_str)

        # Converting the input date to UTC
        input_date_utc = input_date.astimezone(pytz.UTC)

        # Getting the current date in UTC
        current_date_utc = datetime.now(timezone.utc)

        # Calculating the number of days difference
        Days = (current_date_utc - input_date_utc).days

    except:
        counter+=1
        continue
    
    try:
        # Ignore Nominations for now
        if BillType == "Nomination":
            continue  
        
        API_RETURN_COUNT += 1
        if API_RETURN_COUNT > 500:
            API_RETURN_COUNT = Adjust_And_Pause()

        UpdatedDate,IntroducedDate,Party,OriginChamber,policyArea,Title = extract_bill_info(Congress, Number, BillType, API_RETURN_COUNT)

        API_RETURN_COUNT += 1
        if API_RETURN_COUNT > 500:
            API_RETURN_COUNT = Adjust_And_Pause()
        Summary = retrieve_summary(Congress, Number, BillType, API_RETURN_COUNT)

        API_RETURN_COUNT += 1
        if API_RETURN_COUNT > 500:
            API_RETURN_COUNT = Adjust_And_Pause()
        PDF_URL = retrieve_pdf_url(Congress, Number, BillType, API_RETURN_COUNT)

        
        # The ID is the Congress number followed by -O "Origin Chamber" and -C "Chamber"
        ID = str(Congress) + '-' + str(Number) + '-O' + OriginChamber[0] + '-C' + Chamber[0]

        if Party == "D":
            Party = "Democrats"
        elif Party == "R":
            Party = "Republicans"
        else:
            Party = "Independent"  

        if Title != '' and Congress != '' and Number != '' and UpdatedDate != '' and IntroducedDate != '' and Summary != '' and PDF_URL != '':
            CongressionalBillInfoOBJ = CongressionalBillInfo(ID = ID, Category = Category, Session = Session, OriginChamber = OriginChamber, Result=Result, Title = Title, Summary = Summary, PolicyArea = policyArea, IntroducedDate = IntroducedDate, UpdatedDate = UpdatedDate, PDF_URL = PDF_URL, Party = Party, Chamber = Chamber, Days = Days, Aye = aye_list, Nay = nay_list, Not_Voting = not_voting_list, Present = present_list,DemocratAyeCount = DemocratAyeCount, RepublicanAyeCount = RepublicanAyeCount, RepublicanNayCount = RepublicanNayCount,DemocratNayCount = DemocratNayCount, DemocratNotVotingCount = DemocratNotVotingCount, RepublicanNotVotingCount = RepublicanNotVotingCount)
            BillOBJList.append(CongressionalBillInfoOBJ)
    except:
        print("Error in Second Try Statement")

#### Delete the Congress Gov Folders

### Read and Write from DynamoDB

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

Query for the politiciansDB so that corresponding voting information can be added to the db

In [ ]:
def query_Politicians_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('PoliticiansDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items
Current_DB_Politicians = query_Politicians_DB()

In [ ]:
len(BillOBJList)

Create lists of republican, democrat, and independents by alternateids. This will be used when going through legislation to categorize each vote properly.

In [ ]:
republican_alt_id_list = []
democrat_alt_id_list = []
independent_alt_id_list = []

for politician in Current_DB_Politicians:
    
    if politician['Party'] == "Republican":
        republican_alt_id_list.append(politician['alternateID'])
    elif politician['Party'] == "Democratic":
        democrat_alt_id_list.append(politician['alternateID'])
    elif politician['Party'] == "Independent":
        independent_alt_id_list.append(politician['alternateID'])

Create a class that also contains independent voting counts along with the 'Name_Party_State" dictionary lists where the key is the politician alternateID and then the rest is a list of the corresponding politician's name, state, and party they are associated with.

In [2]:
class CongressionalBillInfoWithExtendedVoteInfo:
    def __init__ (self, ID, Title, Summary, IntroducedDate, UpdatedDate, PDF_URL, Party, PolicyArea, Category, Session, Chamber,OriginChamber, Days, Aye, Nay, Not_Voting, Aye_Name_Party_State, Nay_Name_Party_State, Not_Voting_Name_Party_State, Present, Present_Name_Party_State, DemocratAyeCount, RepublicanAyeCount, IndependentAyeCount, RepublicanNayCount, DemocratNayCount, IndependentNayCount, DemocratNotVotingCount, RepublicanNotVotingCount, IndependentNotVotingCount, Result):
        self.ID = ID
        self.Title = Title
        self.Summary = Summary
        self.IntroducedDate = IntroducedDate
        self.UpdatedDate = UpdatedDate
        self.PDF_URL = PDF_URL
        self.Party = Party
        self.PolicyArea = PolicyArea
        self.Category = Category
        self.Chamber = Chamber
        self.OriginChamber = OriginChamber
        self.Session = Session
        self.Days = Days
        self.Aye = Aye
        self.Nay = Nay
        self.Not_Voting = Not_Voting
        self.Aye_Name_Party_State = Aye_Name_Party_State
        self.Nay_Name_Party_State = Nay_Name_Party_State
        self.Not_Voting_Name_Party_State = Not_Voting_Name_Party_State
        self.Present_Name_Party_State = Present_Name_Party_State
        self.Present = Present
        self.DemocratAyeCount = DemocratAyeCount
        self.RepublicanAyeCount = RepublicanAyeCount
        self.IndependentAyeCount = IndependentAyeCount
        self.DemocratNayCount = DemocratNayCount
        self.RepublicanNayCount = RepublicanNayCount
        self.IndependentNayCount = IndependentAyeCount
        self.DemocratNotVotingCount = DemocratNotVotingCount
        self.RepublicanNotVotingCount = RepublicanNotVotingCount
        self.IndependentNotVotingCount = IndependentNotVotingCount
        self.Result = Result

In [ ]:
index_counter = 0
Full_Bill_OBJ_List = []

for obj in BillOBJList:
    
    Aye_Name_Party_State_Dictionary = {}
    Nay_Name_Party_State_Dictionary = {}
    Not_Voting_Name_Party_State_Dictionary = {}
    Present_Name_Party_State_Dictionary = {}
    
    for pol_id in obj.Aye:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['alternateID']):
                Aye_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break
        
    for pol_id in obj.Nay:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['alternateID']):
                Nay_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break
    
    for pol_id in obj.Not_Voting:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['alternateID']):
                Not_Voting_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break
    
    for pol_id in obj.Present:
        for full_pol_info_from_DB in Current_DB_Politicians:
            if (pol_id == full_pol_info_from_DB['alternateID']):
                Present_Name_Party_State_Dictionary[full_pol_info_from_DB['alternateID']] = [full_pol_info_from_DB['Name'],full_pol_info_from_DB['Party'],full_pol_info_from_DB['State']]
                break

    DemocratAyeCount = 0
    RepublicanAyeCount = 0
    IndependentAyeCount = 0

    DemocratNayCount = 0
    RepublicanNayCount = 0
    IndependentNayCount = 0

    DemocratNotVotingCount = 0
    RepublicanNotVotingCount = 0
    IndependentNotVotingCount = 0

    for politician_id in obj.Aye:

        if politician_id in republican_alt_id_list:
            RepublicanAyeCount+=1
        elif politician_id in democrat_alt_id_list:
            DemocratAyeCount+=1
        elif politician_id in independent_alt_id_list:
            IndependentAyeCount+=1
    
    BillOBJList[index_counter].RepublicanAyeCount = RepublicanAyeCount
    BillOBJList[index_counter].DemocratAyeCount = DemocratAyeCount
    BillOBJList[index_counter].IndependentAyeCount = IndependentAyeCount

    for politician_id in obj.Nay:

        if politician_id in republican_alt_id_list:
            RepublicanNayCount+=1
        elif politician_id in democrat_alt_id_list:
            DemocratNayCount+=1
        elif politician_id in independent_alt_id_list:
            IndependentNayCount+=1
    
    BillOBJList[index_counter].RepublicanNayCount = RepublicanNayCount
    BillOBJList[index_counter].DemocratNayCount = DemocratNayCount
    BillOBJList[index_counter].IndependentNayCount = IndependentNayCount

    for politician_id in obj.Not_Voting:

        if politician_id in republican_alt_id_list:
            RepublicanNotVotingCount+=1
        elif politician_id in democrat_alt_id_list:
            DemocratNotVotingCount+=1
        elif politician_id in independent_alt_id_list:
            IndependentNotVotingCount+=1        

    BillOBJList[index_counter].RepublicanNotVotingCount = RepublicanNotVotingCount
    BillOBJList[index_counter].DemocratNotVotingCount = DemocratNotVotingCount
    BillOBJList[index_counter].IndependentNotVotingCount = IndependentNotVotingCount
    
    # Update the Days based on Updated Date
    current_date = datetime.now()
    
    
    # print(LegislationJSONList[index_counter]['RepublicanAyeCount'])
    
    index_counter+=1
    
    CongressionalBillInfoWithExtendedVoteInfoOBJ = CongressionalBillInfoWithExtendedVoteInfo(ID = obj.ID, 
                                                                                             Category = obj.Category, 
                                                                                             Session = obj.Session, 
                                                                                             OriginChamber = obj.OriginChamber, 
                                                                                             Result= obj.Result, 
                                                                                             Title = obj.Title, 
                                                                                             Summary = obj.Summary, 
                                                                                             PolicyArea = obj.PolicyArea, 
                                                                                             IntroducedDate = obj.IntroducedDate, 
                                                                                             UpdatedDate = obj.UpdatedDate, 
                                                                                             PDF_URL = obj.PDF_URL, 
                                                                                             Party = obj.Party, 
                                                                                             Chamber = obj.Chamber, 
                                                                                             Days = obj.Days, 
                                                                                             Aye = obj.Aye,
                                                                                             Nay = obj.Nay,
                                                                                             Not_Voting = obj.Not_Voting, 
                                                                                             Aye_Name_Party_State = Aye_Name_Party_State_Dictionary, 
                                                                                             Nay_Name_Party_State = Nay_Name_Party_State_Dictionary, 
                                                                                             Not_Voting_Name_Party_State = Not_Voting_Name_Party_State_Dictionary,
                                                                                             Present = obj.Present,
                                                                                             Present_Name_Party_State = Present_Name_Party_State_Dictionary,
                                                                                             DemocratAyeCount = DemocratAyeCount, 
                                                                                             RepublicanAyeCount = RepublicanAyeCount, 
                                                                                             IndependentAyeCount = IndependentAyeCount,
                                                                                             RepublicanNayCount = RepublicanNayCount, 
                                                                                             DemocratNayCount = DemocratNayCount,
                                                                                             IndependentNayCount = IndependentNayCount,
                                                                                             DemocratNotVotingCount = DemocratNotVotingCount,
                                                                                             RepublicanNotVotingCount = RepublicanNotVotingCount,
                                                                                             IndependentNotVotingCount = IndependentNotVotingCount)
    Full_Bill_OBJ_List.append(CongressionalBillInfoWithExtendedVoteInfoOBJ)

In [ ]:
def write_to_CongressionalBillsDB(CongressionalBillInfoOBJ, dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")

    table = dynamodb.Table('CongressionalBillsDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.put_item(
       Item={
            'id': CongressionalBillInfoOBJ.ID,
            'Title': CongressionalBillInfoOBJ.Title,
            'Summary': CongressionalBillInfoOBJ.Summary,
            'IntroducedDate': CongressionalBillInfoOBJ.IntroducedDate,
            'UpdatedDate': CongressionalBillInfoOBJ.UpdatedDate,
            'PDF_URL': CongressionalBillInfoOBJ.PDF_URL,
            'Party': CongressionalBillInfoOBJ.Party,
            'PolicyArea': CongressionalBillInfoOBJ.PolicyArea,
            'Chamber': CongressionalBillInfoOBJ.Chamber,
            'OriginChamber': CongressionalBillInfoOBJ.OriginChamber,
            'Days': CongressionalBillInfoOBJ.Days,
            'Aye' : CongressionalBillInfoOBJ.Aye,
            'Nay' : CongressionalBillInfoOBJ.Nay,
            'Not_Voting' : CongressionalBillInfoOBJ.Not_Voting,
            'Aye_Name_Party_State': CongressionalBillInfoOBJ.Aye_Name_Party_State,
            'Nay_Name_Party_State': CongressionalBillInfoOBJ.Nay_Name_Party_State,
            'Not_Voting_Name_Party_State': CongressionalBillInfoOBJ.Not_Voting_Name_Party_State,
            'Present' : CongressionalBillInfoOBJ.Present,
            'Present_Name_Party_State' : CongressionalBillInfoOBJ.Present_Name_Party_State,
            'DemocratAyeCount' : CongressionalBillInfoOBJ.DemocratAyeCount,
            'RepublicanAyeCount' : CongressionalBillInfoOBJ.RepublicanAyeCount,
            'IndependentAyeCount' : CongressionalBillInfoOBJ.IndependentAyeCount,
            'RepublicanNayCount' : CongressionalBillInfoOBJ.RepublicanNayCount,
            'DemocratNayCount' : CongressionalBillInfoOBJ.DemocratNayCount,
            'IndependentNayCount' : CongressionalBillInfoOBJ.IndependentNayCount,
            'DemocratNotVotingCount' : CongressionalBillInfoOBJ.DemocratNotVotingCount,
            'RepublicanNotVotingCount' : CongressionalBillInfoOBJ.RepublicanNotVotingCount,
            'IndependentNotVotingCount': CongressionalBillInfoOBJ.IndependentNotVotingCount,
            'Result': CongressionalBillInfoOBJ.Result,
            'Session': CongressionalBillInfoOBJ.Session,
            'Category': CongressionalBillInfoOBJ.Category
        }
    )
    return response

In [ ]:
for Bill in Full_Bill_OBJ_List:
    write_to_CongressionalBillsDB(Bill)

### Write all Politician Information including Votes to S3 based on CongressionalBillsDB

In [4]:
import requests
import re
import ast
import operator
import regex
import os, shutil
import urllib.request
from PIL import Image
import pickle
import time
import sys
import gzip
import os

In [5]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
import json
import pandas as pd
from datetime import datetime

#### Read all the legislation from the Congressional Bill DB

In [9]:
def query_Congress_Leg_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('CongressionalBillsDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items

Prior_Voted_Legistlation = query_Congress_Leg_DB()
Adjusted_Prior_Voted_Legistlation = Prior_Voted_Legistlation.copy()

The Code below takes a legislation dictionary object and finds if there are any "Democratic" mentions and updates them to say "Democrat" and writes again to CongressionalBills DB

In [10]:
def write_to_CongressionalBillsDB_Dictionary(CongressionalBillInfoDict, dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")

    table = dynamodb.Table('CongressionalBillsDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.put_item(
       Item=CongressionalBillInfoDict
    )
    return response

Update the Days based on the current date and the UpdatedDate Time and change the chamber text from 'h' and 's' to House and Senate

In [11]:
for leg in Adjusted_Prior_Voted_Legistlation:
    current_date = datetime.now()
    
    updated_date = leg['UpdatedDate'] 
    prior_date = datetime.strptime(updated_date, '%Y-%m-%d')

    # Calculate the number of days between the current date and the prior date
    days_difference = (current_date - prior_date).days
    leg['Days'] = days_difference
    
    if leg['Chamber'] == 'h':
        leg['Chamber'] = 'House'
    elif leg['Chamber'] == 's':
        leg['Chamber'] = 'Senate'
     
    write_to_CongressionalBillsDB_Dictionary(leg)

#### Read all the Politicians stored in DynamoDB

In [ ]:
def query_Politicians_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('PoliticiansDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items
Current_DB_Politicians = query_Politicians_DB()

In [ ]:
Politician_Party_Dictionary = {}
for politician in Current_DB_Politicians:
    Politician_Party_Dictionary[politician['alternateID']] = politician['Party']

#### Write all the politician information with the corresponding legislation information on each voted bill to a JSON file and then save to S3

Create a folder for saving JSON files to

In [ ]:
import os
# Specify the directory you want to clear
folder_path = 'PoliticianJSONFolder-OrderedLegislation/'
# Create the nested folders
os.makedirs(folder_path, exist_ok=True)

In [ ]:
for politician in Current_DB_Politicians:
    
    pol_alt_id = politician['alternateID']
    print(pol_alt_id)
    Party_of_Politician_In_Primary_For_Loop = politician['Party']
    
    pol_aye_vote_dict = {}
    pol_nay_vote_dict = {}
    pol_no_vote_dict = {}
    pol_present_vote_dict = {}
    
    Pol_Sides_With_Republicans = 0
    Pol_Sides_With_Democrats = 0
    Pol_Did_Not_Vote = 0
    Pol_Present = 0
    Total_Votes_Aye_Nay_Present_Not_Voting = 0

    for legislation in Adjusted_Prior_Voted_Legistlation:
        
        LegislationRepublicanAyeCount = 0
        LegislationRepublicanNayCount = 0
        LegislationDemocratAyeCount = 0
        LegislationDemocratNayCount = 0
        
        Politician_Vote_For_Specific_Legislation = ""

        for aye_legislation_pol in legislation['Aye_Name_Party_State'].keys():
            
            
            Pol_Party = Politician_Party_Dictionary[aye_legislation_pol]
            if Pol_Party == "Republican":
                LegislationRepublicanAyeCount+=1
            elif Pol_Party == "Democrat":
                LegislationDemocratAyeCount+=1

            if aye_legislation_pol == pol_alt_id:
                temp_legislation = legislation.copy()
                Politician_Vote_For_Specific_Legislation = "Aye"
                Total_Votes_Aye_Nay_Present_Not_Voting+=1
                temp_legislation['Aye'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Aye_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Present'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                pol_aye_vote_dict[legislation['id']] = temp_legislation
                # break
        
        for nay_legislation_pol in legislation['Nay_Name_Party_State'].keys():
            
            Pol_Party = Politician_Party_Dictionary[nay_legislation_pol]
            if Pol_Party == "Republican":
                LegislationRepublicanNayCount+=1
            elif Pol_Party == "Democrat":
                LegislationDemocratNayCount+=1
            
            if nay_legislation_pol == pol_alt_id:
                temp_legislation = legislation.copy()
                Politician_Vote_For_Specific_Legislation = "Nay"
                Total_Votes_Aye_Nay_Present_Not_Voting+=1
                temp_legislation['Aye'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Aye_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Present'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                pol_nay_vote_dict[legislation['id']] = temp_legislation
                # break
        
        for not_voting_legislation_pol in legislation['Not_Voting_Name_Party_State'].keys():
            if not_voting_legislation_pol == pol_alt_id:
                temp_legislation = legislation.copy()
                Politician_Vote_For_Specific_Legislation = "Not Voting"
                Pol_Did_Not_Vote+=1
                Total_Votes_Aye_Nay_Present_Not_Voting+=1
                temp_legislation['Aye'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Aye_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Present'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                pol_no_vote_dict[legislation['id']] = temp_legislation
                # break
                
        for present_voting_legislation_pol in legislation['Present_Name_Party_State'].keys():
            if present_voting_legislation_pol == pol_alt_id:
                temp_legislation = legislation.copy()
                Politician_Vote_For_Specific_Legislation = "Present"
                Pol_Present+=1
                Total_Votes_Aye_Nay_Present_Not_Voting+=1
                temp_legislation['Aye'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Aye_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Nay_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Not_Voting_Name_Party_State'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                temp_legislation['Present'] = "Empty for Individual Politician S3 JSON Retrieval Purposes"
                pol_present_vote_dict[legislation['id']] = temp_legislation
                # break
                
        # Determine the number of times that a politician votes with a specific party
        if Politician_Vote_For_Specific_Legislation == "Aye" and Party_of_Politician_In_Primary_For_Loop == "Republican":
            
            if LegislationRepublicanAyeCount >= LegislationRepublicanNayCount:
                Pol_Sides_With_Republicans+=1
            else:
                Pol_Sides_With_Democrats+=1
                
        elif Politician_Vote_For_Specific_Legislation == "Aye" and Party_of_Politician_In_Primary_For_Loop == "Democrat":
            
            if LegislationDemocratAyeCount >= LegislationDemocratNayCount:
                Pol_Sides_With_Democrats+=1
            else:
                Pol_Sides_With_Republicans+=1
                
        elif Politician_Vote_For_Specific_Legislation == "Nay" and Party_of_Politician_In_Primary_For_Loop == "Republican":
            
            if LegislationRepublicanNayCount >= LegislationRepublicanAyeCount:
                Pol_Sides_With_Republicans+=1
            else:
                Pol_Sides_With_Democrats+=1
                
        elif Politician_Vote_For_Specific_Legislation == "Nay" and Party_of_Politician_In_Primary_For_Loop == "Democrat":
            
            if LegislationDemocratNayCount >= LegislationDemocratAyeCount:
                Pol_Sides_With_Democrats+=1
            else:
                Pol_Sides_With_Republicans+=1
                
        elif Politician_Vote_For_Specific_Legislation == "Aye" and Party_of_Politician_In_Primary_For_Loop == "Independent":
            
            if LegislationDemocratAyeCount >= LegislationDemocratNayCount:
                Pol_Sides_With_Democrats+=1
            elif LegislationRepublicanAyeCount >= LegislationRepublicanNayCount:
                Pol_Sides_With_Republicans+=1
        
        elif Politician_Vote_For_Specific_Legislation == "Nay" and Party_of_Politician_In_Primary_For_Loop == "Independent":
            
            if LegislationDemocratNayCount >= LegislationDemocratAyeCount:
                Pol_Sides_With_Democrats+=1
            elif LegislationRepublicanNayCount >= LegislationRepublicanAyeCount:
                Pol_Sides_With_Republicans+=1
            
    #Determine the overall percentage that a politician votes by Republican or Democrat
    PercentVoteWithRepublicans = Pol_Sides_With_Republicans/Total_Votes_Aye_Nay_Present_Not_Voting
    PercentVoteWithDemocrats = Pol_Sides_With_Democrats/Total_Votes_Aye_Nay_Present_Not_Voting
    PercentDidNotVote = Pol_Did_Not_Vote/Total_Votes_Aye_Nay_Present_Not_Voting
    PercentPresent = Pol_Present/Total_Votes_Aye_Nay_Present_Not_Voting
    
    
    politician['PercentVoteWithRepublicans'] = PercentVoteWithRepublicans
    politician['PercentVoteWithDemocrats'] = PercentVoteWithDemocrats
    politician['PercentDidNotVote'] = PercentDidNotVote
    politician['PercentPresent'] = PercentPresent
    
    current_date = datetime.now()
    
    # Order the Politician Aye Votes based on last updated date
    days_diff_array = []
    leg_id_list = []
    for aye_vote in pol_aye_vote_dict:
        updated_date = pol_aye_vote_dict[aye_vote]['UpdatedDate']
        prior_date = datetime.strptime(updated_date, '%Y-%m-%d')

        # Calculate the number of days between the current date and the prior date
        days_difference = (current_date - prior_date).days
        days_diff_array.append(days_difference)
        leg_id_list.append(aye_vote)
    
    sorted_pairs = sorted(zip(days_diff_array, leg_id_list))
    # Extract the elements of list b in the sorted order
    sorted_ids = [element for _, element in sorted_pairs]
    
    pol_aye_vote_dict_with_ordered_index_keys = {}
    index_counter = 0
    for leg_id in sorted_ids:
        pol_aye_vote_dict_with_ordered_index_keys[index_counter] = pol_aye_vote_dict[leg_id]
        index_counter+=1
        
    # Order the Politician Nay Votes based on last updated date
    days_diff_array = []
    leg_id_list = []
    for nay_vote in pol_nay_vote_dict:
        updated_date = pol_nay_vote_dict[nay_vote]['UpdatedDate']
        prior_date = datetime.strptime(updated_date, '%Y-%m-%d')

        # Calculate the number of days between the current date and the prior date
        days_difference = (current_date - prior_date).days
        days_diff_array.append(days_difference)
        leg_id_list.append(nay_vote)
    
    sorted_pairs = sorted(zip(days_diff_array, leg_id_list))
    # Extract the elements of list b in the sorted order
    sorted_ids = [element for _, element in sorted_pairs]
    
    pol_nay_vote_dict_with_ordered_index_keys = {}
    index_counter = 0
    for leg_id in sorted_ids:
        pol_nay_vote_dict_with_ordered_index_keys[index_counter] = pol_nay_vote_dict[leg_id]
        index_counter+=1
    
    # Order the Politician Not Voting Votes based on last updated date
    days_diff_array = []
    leg_id_list = []
    for no_vote in pol_no_vote_dict:
        updated_date = pol_no_vote_dict[no_vote]['UpdatedDate']
        prior_date = datetime.strptime(updated_date, '%Y-%m-%d')

        # Calculate the number of days between the current date and the prior date
        days_difference = (current_date - prior_date).days
        days_diff_array.append(days_difference)
        leg_id_list.append(no_vote)
    
    sorted_pairs = sorted(zip(days_diff_array, leg_id_list))
    # Extract the elements of list b in the sorted order
    sorted_ids = [element for _, element in sorted_pairs]
    
    pol_no_vote_dict_with_ordered_index_keys = {}
    index_counter = 0
    for leg_id in sorted_ids:
        pol_no_vote_dict_with_ordered_index_keys[index_counter] = pol_no_vote_dict[leg_id]
        index_counter+=1
    
    
    politician['AyeVotedBills'] = pol_aye_vote_dict_with_ordered_index_keys
    politician['NayVotedBills'] = pol_nay_vote_dict_with_ordered_index_keys
    politician['NotVotedBills'] = pol_no_vote_dict_with_ordered_index_keys
    
    # Specify the filename
    filename = "PoliticianJSONFolder-OrderedLegislation/" + politician['id'] + '.json'
    # Convert Decimal to string during serialization

    # Open the file in write mode ('w') and write the dictionary
    with open(filename, 'w') as file:
        json.dump(politician, file, default = str, indent=4)  # Use 'indent' for pretty printing

    bucket_name = "moderateapps3bucket01959-dev"
    file_name_in_s3 = "public/" + filename

    # json_str = json.dumps(politician, default=str)
    # print(type(json_str))
    # json_data = json.loads(json_str)

    s3 = boto3.client('s3')
    with open(filename, "rb") as f:
        bucket = "moderateapps3bucket01959-dev"
        save_string = "public/" + filename
        s3.upload_fileobj(f, bucket, save_string)


#### Delete All the JSON files in the Folder

In [ ]:
# Specify the directory you want to clear
folder_path = 'PoliticianJSONFolder-OrderedLegislation/'

# Loop through all files in the directory
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        # Check if it is a file and not a directory
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path) # This will delete the file
        # Uncomment below if you also want to remove subdirectories
        # elif os.path.isdir(file_path):
        #     shutil.rmtree(file_path) # Be careful with this, it deletes directories
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')
